In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
/kaggle/input/models/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5
/kaggle/input/infrared-solar-modules/LICENSE
/kaggle/input/infrared-solar-modules/README.md
/kaggle/input/infrared-solar-modules/2020-02-14_InfraredSolarModules/InfraredSolarModules/module_metadata.json


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import cv2 as cv
import random
import tensorflow as tf
import numpy as np

np.random.seed(1)
tf.random.set_seed(1)

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, MaxPool2D, Dropout
from tensorflow.keras.models import Sequential


In [ ]:
import pandas as pd
import json
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

In [ ]:
with open('/kaggle/input/infrared-solar-modules/2020-02-14_InfraredSolarModules/InfraredSolarModules/module_metadata.json', 'r') as file:
    data = json.load(file)

In [ ]:
df = pd.DataFrame(data).T.reset_index(drop=True)

In [ ]:
def load_images(dataframe, base_path="/kaggle/input/infrared-solar-modules/2020-02-14_InfraredSolarModules/InfraredSolarModules/"):
    images = []
    labels = []
    i = 0
    for idx, row in dataframe.iterrows():
        i+=1
        img_path = base_path + row['image_filepath']
        img = Image.open(img_path).convert('RGB')
        img = img.resize((128, 128))  # Resize the image
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # Normalize to [0, 1]
        images.append(img_array)
        labels.append(row['anomaly_class'])
        #print(f'{i} Done out of {dataframe.shape[0]}')     # enable if you need to check the itrs of reading
    return np.array(images), np.array(labels)

In [ ]:
# Split data into features and target || Convert labels to categorical codes
X = df['image_filepath']
y = pd.Categorical(df['anomaly_class']).codes 

print(len(X))

In [ ]:
# Split the dataset into training (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [ ]:
# Convert Series to DataFrame to reuse the load_images function
train_df = pd.DataFrame({'image_filepath': X_train, 'anomaly_class': y_train})
val_df = pd.DataFrame({'image_filepath': X_val, 'anomaly_class': y_val})
test_df = pd.DataFrame({'image_filepath': X_test, 'anomaly_class': y_test})

print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

In [10]:
# Load and preprocess images
train_images, train_labels = load_images(train_df)
print("Train Read Done..!")
val_images, val_labels = load_images(val_df)
print("Validation Read Done..!")
test_images, test_labels = load_images(test_df)
print("Test Read Done..!")

Train Read Done..!
Validation Read Done..!
Test Read Done..!


In [11]:
# Optionally, convert labels to categorical format if using categorical crossentropy
train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)
test_labels = to_categorical(test_labels)

In [12]:
import cv2 as cv 
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from keras.layers import Bidirectional
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.utils import to_categorical
import itertools
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, VGG19,ResNet50,ResNet101, ResNet152, InceptionV3, MobileNet, DenseNet121, DenseNet169, DenseNet201, NASNetMobile, Xception


In [13]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.utils import to_categorical
import cv2 as cv
import tensorflow as tf



In [14]:

# Custom Callback for logging after each epoch
class EpochLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}: Train Accuracy: {logs['accuracy']:.4f}, Train Loss: {logs['loss']:.4f}")

# Function to reshape data for uniform input
def reshape_data(data, new_shape):
    reshaped_data = np.zeros((data.shape[0],) + new_shape)
    for i in range(data.shape[0]):
        reshaped_data[i] = cv.resize(data[i], new_shape[:2])  # Resize images to (height, width)
    return reshaped_data

# Function to create a CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(12, activation='softmax')  # 12 classes for your dataset
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Base model loading paths

densenet169_weights_path = '/kaggle/input/models/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
#densenet201_weights_path = '/kaggle/input/model2/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Load the base models (for feature extraction, no fine-tuning)
base_models = [
   
    VGG19(weights=densenet169_weights_path , include_top=False, input_shape=(224, 224, 3)),
]

# Hyperparameters for training
learning_rates = [0.001]
epochs_list = [80]
input_shape = (128, 128, 3)
results = []

# Data augmentation setup
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augmented training generator (assume `train_images` and `train_labels` are preloaded)
train_generator = datagen.flow(train_images, train_labels, batch_size=32)

# Original dataset size
N_train = train_images.shape[0]  # Number of training images
batch_size = 32  # Your defined batch size

# Calculate batches per epoch
batches_per_epoch = N_train // batch_size

# Total augmented images per epoch
augmented_images_per_epoch = batches_per_epoch * batch_size
print(f"Augmented images generated per epoch: {augmented_images_per_epoch}")


Augmented images generated per epoch: 12000


In [15]:
# Iterate over each base model, learning rate, and epoch configuration
for base_model in base_models:
    print(f"Using base model: {base_model.name}")
    for lr in learning_rates:
        print(f"Learning rate: {lr}")
        for epochs in epochs_list:
            print(f"Training for {epochs} epochs...")

            # Create the CNN model (no fine-tuning, just feature extraction)
            model = create_cnn_model(input_shape=(128, 128, 3))
            print("CNN model created successfully.")

            # Reshape the data
            reshaped_x_train = reshape_data(train_images, (128, 128, 3))  # Keep data in 128x128
            reshaped_x_test = reshape_data(test_images, (128, 128, 3))
            print(f"Data reshaped to {reshaped_x_train.shape}.")

            # Show the size of the augmented image for debugging
            sample_augmented_image = next(train_generator)[0][0]
            print(f"Sample augmented image shape: {sample_augmented_image.shape}")

            # Train the model using augmented data
            history = model.fit(train_generator, epochs=epochs, validation_data=(reshaped_x_test, test_labels),
                                callbacks=[EpochLogger()], verbose=0)

            # Evaluate the model on the test set
            test_loss, test_accuracy = model.evaluate(reshaped_x_test, test_labels, verbose=0)
            print(f"Model trained successfully. Test accuracy: {test_accuracy:.4f}")
            
            # Save the results
            results.append([base_model.name, epochs, lr, test_accuracy])

# After all models have been trained
print("Training complete. Saving results to Excel...")

# Save results to Excel
df = pd.DataFrame(results, columns=['Model', 'Epochs', 'Learning Rate', 'Test Accuracy'])
df.to_excel('model_accuracies_with_augmentation.xlsx', index=False)
print("Results saved successfully.")


Using base model: vgg19
Learning rate: 0.001
Training for 80 epochs...
CNN model created successfully.
Data reshaped to (12000, 128, 128, 3).
Sample augmented image shape: (128, 128, 3)


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1728529157.172584      96 service.cc:145] XLA service 0x78d6d8004dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728529157.172660      96 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1728529159.793010      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1: Train Accuracy: 0.4977, Train Loss: 1.8442
Epoch 2: Train Accuracy: 0.4993, Train Loss: 1.7481
Epoch 3: Train Accuracy: 0.5002, Train Loss: 1.7267
Epoch 4: Train Accuracy: 0.5005, Train Loss: 1.7130
Epoch 5: Train Accuracy: 0.5016, Train Loss: 1.6972
Epoch 6: Train Accuracy: 0.5013, Train Loss: 1.6917
Epoch 7: Train Accuracy: 0.5032, Train Loss: 1.6788
Epoch 8: Train Accuracy: 0.5002, Train Loss: 1.6676
Epoch 9: Train Accuracy: 0.5048, Train Loss: 1.6642
Epoch 10: Train Accuracy: 0.5063, Train Loss: 1.6398
Epoch 11: Train Accuracy: 0.5123, Train Loss: 1.6182
Epoch 12: Train Accuracy: 0.5214, Train Loss: 1.5790
Epoch 13: Train Accuracy: 0.5342, Train Loss: 1.5385
Epoch 14: Train Accuracy: 0.5413, Train Loss: 1.5206
Epoch 15: Train Accuracy: 0.5469, Train Loss: 1.4965
Epoch 16: Train Accuracy: 0.5502, Train Loss: 1.4904
Epoch 17: Train Accuracy: 0.5579, Train Loss: 1.4628
Epoch 18: Train Accuracy: 0.5563, Train Loss: 1.4773
Epoch 19: Train Accuracy: 0.5616, Train Loss: 1.4598
Ep